In [15]:
# setups

import os
import json

import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role, Session


In [16]:
sess = Session()
bucket = sess.default_bucket()
prefix = "sagemaker/pytorch-toneclone"
role = get_execution_role()

In [17]:
model = PyTorchModel(
    entry_point="spectrogramPANN_inference.py",
    role=role,
    model_data='s3://toneclone-bucket/Models/pann.tar.gz',
    framework_version="1.11.0",
    py_version="py38",
)

In [18]:
from sagemaker.serializers import NumpySerializer
from sagemaker.deserializers import JSONDeserializer

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.large',
    serializer=NumpySerializer(),
    deserializer=JSONDeserializer(),
    endpoint_name='ToneClonePANN'
)

---------------!

In [19]:
import numpy as np

test_single = np.load('single_spectrogram.npy')
test_multiple = np.load('spectrograms.npy')

In [20]:
response_single = predictor.predict(test_single)

In [21]:
response_single

{'Segment 1 (00:00 - 00:10)': [['fuzz'],
  {'overdrive': 1.0524400795297595e-11,
   'distortion': 5.66529408021349e-13,
   'fuzz': 1.0,
   'tremolo': 2.042296019799661e-12,
   'phaser': 4.601127963875518e-13,
   'flanger': 2.339441574489287e-12,
   'chorus': 9.353867369554791e-08,
   'delay': 6.289821441463062e-10,
   'hall_reverb': 3.6636749948908143e-13,
   'plate_reverb': 1.8435571125241523e-10,
   'octaver': 4.195149358565686e-06,
   'auto_filter': 0.12043364346027374}]}

In [22]:
response_multiple = predictor.predict(test_multiple)

In [23]:
response_multiple

{'Segment 1 (00:00 - 00:10)': [['fuzz'],
  {'overdrive': 1.0524400795297595e-11,
   'distortion': 5.665316306358026e-13,
   'fuzz': 1.0,
   'tremolo': 2.04229211667184e-12,
   'phaser': 4.601118748157051e-13,
   'flanger': 2.3394461281384116e-12,
   'chorus': 9.353850316529133e-08,
   'delay': 6.289821441463062e-10,
   'hall_reverb': 3.663667947576693e-13,
   'plate_reverb': 1.8435536430772004e-10,
   'octaver': 4.195152996544493e-06,
   'auto_filter': 0.12043361365795135}],
 'Segment 2 (00:10 - 00:20)': [['fuzz'],
  {'overdrive': 3.4971338325195944e-11,
   'distortion': 1.2244222129254023e-11,
   'fuzz': 1.0,
   'tremolo': 1.2913538511086231e-12,
   'phaser': 5.257128825146562e-13,
   'flanger': 2.8291929562995755e-11,
   'chorus': 1.2545079641768098e-08,
   'delay': 1.3087477768181088e-09,
   'hall_reverb': 4.675357323513651e-10,
   'plate_reverb': 1.6382875401177444e-05,
   'octaver': 7.974717846082058e-07,
   'auto_filter': 0.007228201720863581}],
 'Segment 3 (00:20 - 00:30)': [['f

In [24]:
import boto3
import json
from tempfile import TemporaryFile

outfile = TemporaryFile()

endpoint = 'ToneClonePANN'
 
runtime = boto3.Session().client('sagemaker-runtime')

np.save(outfile, test_multiple)
_ = outfile.seek(0)

In [25]:
response = runtime.invoke_endpoint(EndpointName=endpoint, ContentType='application/x-npy', Body=outfile)

In [26]:
response

{'ResponseMetadata': {'RequestId': '8402c883-d1ca-4ea2-845b-078d6ce48d7a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8402c883-d1ca-4ea2-845b-078d6ce48d7a',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Thu, 27 Mar 2025 03:16:10 GMT',
   'content-type': 'application/json',
   'content-length': '8655',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x7f67915309d0>}

In [27]:
result = json.loads(response['Body'].read().decode())

In [28]:
result

{'Segment 1 (00:00 - 00:10)': [['fuzz'],
  {'overdrive': 1.0524400795297595e-11,
   'distortion': 5.665316306358026e-13,
   'fuzz': 1.0,
   'tremolo': 2.04229211667184e-12,
   'phaser': 4.601118748157051e-13,
   'flanger': 2.3394461281384116e-12,
   'chorus': 9.353850316529133e-08,
   'delay': 6.289821441463062e-10,
   'hall_reverb': 3.663667947576693e-13,
   'plate_reverb': 1.8435536430772004e-10,
   'octaver': 4.195152996544493e-06,
   'auto_filter': 0.12043361365795135}],
 'Segment 2 (00:10 - 00:20)': [['fuzz'],
  {'overdrive': 3.4971338325195944e-11,
   'distortion': 1.2244222129254023e-11,
   'fuzz': 1.0,
   'tremolo': 1.2913538511086231e-12,
   'phaser': 5.257128825146562e-13,
   'flanger': 2.8291929562995755e-11,
   'chorus': 1.2545079641768098e-08,
   'delay': 1.3087477768181088e-09,
   'hall_reverb': 4.675357323513651e-10,
   'plate_reverb': 1.6382875401177444e-05,
   'octaver': 7.974717846082058e-07,
   'auto_filter': 0.007228201720863581}],
 'Segment 3 (00:20 - 00:30)': [['f

In [ ]:
predictor.delete_endpoint()